In [1]:
import torch 
import pandas
# !pip install scipy==1.1.0
# from scipy.misc import imread
import glob
import math
import matplotlib.pyplot as plt
import multiprocessing
import PIL
from PIL import Image
import openslide
import os
from glob import glob
import sys
import numpy as np
%load_ext autoreload
%aimport utils

# import utils

from tqdm.auto import tqdm
from PIL import Image, ImageDraw, ImageFont
import skimage.color as sk_color
import skimage.exposure as sk_exposure
import skimage.feature as sk_feature
import skimage.filters as sk_filters
import skimage.future as sk_future
import skimage.morphology as sk_morphology
import skimage.segmentation as sk_segmentation
import scipy.ndimage.morphology as sc_morph
from enum import Enum
from utils import *

In [2]:
%run utils.py


In [65]:
#Converting the tif into a png for processing int he cores_tiles func 
for idx,path in enumerate(original_file_names):
    print("Done %d out of %d"%(idx,len(original_file_names)))
    imgpath=path
    slide_info(imgpath)
    training_slide_to_image(imgpath,DEST_TRAIN_DI="F:\Data\peso_dataset\scaled_slides_tif",scale_factor=SCALE_FACTOR)

967, 6580), (1483, 3290), (741, 1645), (370, 822))
Level downsamples: (1.0, 2.000009497758529, 4.000018995517058, 8.000374975027377, 16.002706135269115, 32.010808104823255, 64.04322139226954, 128.21197146051162)
Dimensions: (47484, 105289)
Associated images:
Format: generic-tiff
Scaling to 32 ...
Scaling to 32 ...
Opening Slide : F:\Data\peso_dataset\slides_tif\pds_46_HE.tif
Saving image to: F:\Data\peso_dataset\scaled_slides_tif\pds_46_HE.png
Done 31 out of 62

Opening Slide #1: F:\Data\peso_dataset\slides_tif\pds_4_HE.tif
Level count: 8
Level dimensions: ((47484, 105289), (23742, 52644), (11871, 26322), (5935, 13161), (2967, 6580), (1483, 3290), (741, 1645), (370, 822))
Level downsamples: (1.0, 2.000009497758529, 4.000018995517058, 8.000374975027377, 16.002706135269115, 32.010808104823255, 64.04322139226954, 128.21197146051162)
Dimensions: (47484, 105289)
Associated images:
Format: generic-tiff
Scaling to 32 ...
Scaling to 32 ...
Opening Slide : F:\Data\peso_dataset\slides_tif\pds_4_

In [ ]:
#Converting the tif into a png for processing int he cores_tiles func [annotations]
for idx,path in enumerate(annotated_file_names):
    print("Done %d out of %d"%(idx,len(annotated_file_names)))
    imgpath=path
    slide_info(imgpath)
    training_slide_to_image(imgpath,DEST_TRAIN_DI="F:\Data\peso_dataset\scaled_annotated_test_dir",scale_factor=SCALE_FACTOR)

In [3]:
scaled=sorted(glob("F:\\Data\peso_dataset\\scaled_slides_tif\\*.png"))

In [ ]:
%run utils.py
i=11
im_path  = scaled[2]
imgpath = get_slide_path_based_on_PNG_path(im_path,test=True)
dim = slide_to_scaled_pil_image(imgpath,SCALE_FACTOR)
tile_sum=score_tiles(im_path,dimensions=dim[1:],test=True)


In [ ]:
summ,sum_org = generate_top_tile_summaries(tile_sum,im_path)

In [ ]:
for t in tile_sum.top_tiles():
    print(t.tissue_percentage)

    if t.tissue_percentage > .15:

        t.save_tile()

In [4]:
%run utils.py

print(f"annotated_file_names {len(annotated_file_names)}")
print(f"annotated_test_dir {len(annotated_test_dir)}")
print(f"original_file_names {len(original_file_names)}")
print(f"scaled {len(scaled)}")


annotated_file_names 62
annotated_test_dir 62
original_file_names 62
scaled 62


In [9]:
#Creating a dataset of test tiles  
%run utils.py

from tqdm import tqdm
for idx,path in tqdm(enumerate(scaled),total=len(scaled)):
    print("\n#%d##Genertaing tiles for %s"%(idx,path))

    imgpath = get_slide_path_based_on_PNG_path(path,test=True)
    dim = slide_to_scaled_pil_image(imgpath,SCALE_FACTOR)
    tile_sum=score_tiles(path,dimensions=dim[1:],test=True)
    for t in tile_sum.top_tiles():
        # print(t.score)

        if t.score > .15:

            t.save_tile()

  0%|          | 0/62 [00:00<?, ?it/s]
#0##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_100_HE.png
  2%|▏         | 1/62 [00:38<39:08, 38.50s/it]
#1##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_101_HE.png
  3%|▎         | 2/62 [04:20<2:26:24, 146.41s/it]
#2##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_102_HE.png
  5%|▍         | 3/62 [06:44<2:22:52, 145.30s/it]
#3##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_10_HE.png
  6%|▋         | 4/62 [07:11<1:35:19, 98.62s/it] 
#4##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_11_HE.png
  8%|▊         | 5/62 [13:01<2:59:49, 189.29s/it]
#5##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_12_HE.png
 10%|▉         | 6/62 [23:03<5:07:35, 329.57s/it]
#6##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\pds_14_HE.png
 11%|█▏        | 7/62 [33:14<6:26:26, 421.58s/it]
#7##Genertaing tiles for F:\Data\peso_dataset\scaled_slides_tif\p

In [ ]:
# deleting the extra masks
for path in annotated_file_names:
    num = get_num(path)
    matches = [ x for x in annotated_file_names if num == get_num(x)]
    if len(matches) > 1:
        print(f"deleting -- {matches[0]}")
        os.system(f"del -i {matches[0]}")
    print(len(sorted(glob("F:\Data\peso_dataset\\annotated_test_dir\\*")))
)